# Hyperparameter Tuning using HyperDrive

In [21]:
import joblib

from azureml.core import Dataset, Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, randint
from azureml.core import Environment, ScriptRunConfig
import os

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'sleep-health-project-hyperdrive'

exp = Experiment(ws, experiment_name)

In [3]:
cluster_name = "sleep-health-compute"

# Verfiy that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The Sleep Health and Lifestyle Dataset from Kaggle is used to perform a classification task. The data covers a wide range of variables related to sleep and daily habits. In the classification task it should be determined wether a person has a certain sleep disorder or none.

In [4]:
dataset = Dataset.get_by_name(ws, name='Sleep-Health-Dataset')
df = dataset.to_pandas_dataframe()
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# Specify parameter sampler
ps = RandomParameterSampling({
    "learning_rate": choice(0.01, 0.05, 0.1, 0.3, 1),
    "n_estimators": randint(100),
})

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=3,
    slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    compute_target=cluster,
    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=3)

In [5]:
hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the best run's metrics
best_run_metrics = best_run.get_metrics()

print(best_run.id)
print(best_run_metrics)

HD_d38e7295-067c-4c76-afeb-f723f01a39c1_0
{'Learning Rate:': 0.05, 'Number Estimators:': 68, 'Accuracy': 0.9736842105263158}


In [18]:
# Register best model
model = best_run.register_model(model_path="trained_model.pkl",model_name="best_model_gradient_boosting")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
model = Model(ws, "best_model_gradient_boosting")

# Create a deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Define the inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=sklearn_env)

# Deploy the model to AKS
service = Model.deploy(ws, "sleep-health-service", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-07-13 12:25:54+00:00 Creating Container Registry if not exists.
2023-07-13 12:25:56+00:00 Use the existing image.
2023-07-13 12:25:56+00:00 Generating deployment configuration.
2023-07-13 12:25:56+00:00 Submitting deployment to compute.
2023-07-13 12:26:01+00:00 Checking the status of deployment sleep-health-service..
2023-07-13 12:27:54+00:00 Checking the status of inference endpoint sleep-health-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3dd19c5f-293d-4b03-8917-ebd3f5fe93ba
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sleep-health-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image e3373630ac7f428d940dc3ccb8818731.azurecr.io/azureml/azureml_fd72743d916d859b029fea2f65d9c72a locally. Please refer to https://aka.ms/debugimage#service-laun

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3dd19c5f-293d-4b03-8917-ebd3f5fe93ba
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sleep-health-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image e3373630ac7f428d940dc3ccb8818731.azurecr.io/azureml/azureml_fd72743d916d859b029fea2f65d9c72a locally. Pl

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
service.scoring_uri

### Send request to model endpoint

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

